In [ ]:
import pandas as pd

In [ ]:
%config Completer.use_jedi = True

In [ ]:
df_devices = pd.read_csv('../data/devices.csv')
df_notifications = pd.read_csv('../data/notifications.csv')
df_transactions = pd.read_csv('../data/transactions.csv')
df_users = pd.read_csv('../data/users.csv')

display(df_devices.head(5), df_notifications.head(5), df_transactions.head(5), df_users.head(5))

In [ ]:
display(df_devices.info(), df_notifications.info(), df_transactions.info(), df_users.info())

In [ ]:
display(df_devices.describe(), df_notifications.describe(), df_transactions.describe(), df_users.describe())

In [ ]:
## cambiar el nombre a la tabla

df_devices = df_devices.rename(columns={"string_field_0": "brand_device", "string_field_1": "user_id"})
df_notifications = df_notifications.rename(columns={'created_date': 'create_date_notification'})
df_transactions = df_transactions.rename(columns={'created_date': 'create_date_transaction'})
df_users = df_users.rename(columns={'created_date': 'create_date_user'})

display(df_devices, df_notifications, df_transactions, df_users)

### Merge

In [ ]:
pd.set_option('display.max_columns', None)

df = df_users.merge(df_devices, on = 'user_id', how='left')
df = df.merge(df_notifications, on = "user_id", how = "left")
df = df.merge(df_transactions, on = "user_id", how = 'left')
df.head(30)

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
df.shape

In [ ]:
df['create_date_user'] = pd.to_datetime(df['create_date_user'])
df['create_date_transaction'] = pd.to_datetime(df['create_date_transaction'])
df['create_date_notification'] = pd.to_datetime(df['create_date_notification'])
df.info()

### Top 10 paises con más usuarios

In [ ]:
country_users = df.groupby('country')['user_id'].nunique().sort_values(ascending=False).head(10).reset_index()
country_users.columns = ['country', 'unique_users']
country_users

In [ ]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'notebook'  # o 'iframe_connected' o 'svg'

fig_top10_countries = px.bar(
    country_users,
    x="country",
    y="unique_users",
    title="Top 10 usuarios por país"
)

fig_top10_countries.show()

In [ ]:
df.to_csv('../data/df.csv')

In [ ]:
crypto_users = df.groupby('user_settings_crypto_unlocked')['user_id'].nunique().reset_index()
crypto_users

In [ ]:
fig_usuarios_crypto = px.bar(
    crypto_users,
    x="user_settings_crypto_unlocked",
    y="user_id",
    title="Usuarios con cryptomonedas"
)

fig_usuarios_crypto.show()

In [ ]:
df['plan'].unique()

In [ ]:
plan_user = df.groupby('plan')['user_id'].nunique().sort_values(ascending=False).reset_index()
plan_user

In [ ]:
fig_usuarios_plan = px.bar(
    plan_user,
    x="plan",
    y="user_id",
    title="Tipos de plan por uusario"
)

fig_usuarios_plan.show()

In [ ]:
status_noti = df.groupby('status')['user_id'].nunique().reset_index()
df_notifications.shape, status_noti

In [ ]:
channel = df.groupby('channel')['user_id'].nunique().reset_index()
df_notifications.shape, channel

In [ ]:
transactions_type = df.groupby('transactions_type')['user_id'].nunique().sort_values(ascending=False).reset_index()
transactions_type

In [ ]:
transactions_state = df.groupby('transactions_state')['user_id'].count().sort_values(ascending=False).reset_index()
transactions_state

In [ ]:
df.sample()

In [ ]:
df['plan'].value_counts(), df['country'].value_counts().head(10), df['transactions_state'].value_counts()

In [ ]:
df[['amount_usd', 'birth_year']].describe()

In [ ]:
display(df['birth_year'].value_counts().sort_index(), df[df['birth_year'] < 1900])  # valores que no tienen sentido


In [ ]:
df.corr(numeric_only=True)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm')
plt.title("Correlación entre variables numéricas")
plt.show()

In [ ]:
display(
    df.groupby('plan')['amount_usd'].mean().sort_values(ascending=False), 
    df.groupby('country')['num_contacts'].mean().sort_values(ascending=False)
)

In [ ]:
# Transacciones promedio por plan
df_transactions.groupby('user_id').agg({
    'transaction_id': 'count'
}).merge(df_users, on='user_id').groupby('plan')['transaction_id'].mean().sort_values(ascending=False)

In [ ]:
# ¿Los usuarios que recibieron notificaciones transaccionan más?
notified_users = df_notifications['user_id'].unique()
df['received_notification'] = df['user_id'].isin(notified_users)
df.groupby('received_notification')['amount_usd'].mean()

In [ ]:
df.groupby('user_settings_crypto_unlocked')['amount_usd'].mean()

In [ ]:
df.groupby('birth_year')['user_id'].nunique().sort_values(ascending=False).head(10)

In [ ]:
df['age'] = 2025 - df['birth_year']

bins = [0, 17, 24, 34, 44, 54, 64, 120]
labels = ['<18', '18–24', '25–34', '35–44', '45–54', '55–64', '65+']

df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=True)

df.groupby('age_group')['user_id'].nunique().sort_values(ascending=False)


In [ ]:
df